# Getting started with STIsim

STIsim is an agent-based modeling framework for sexually transmitted infections, built on top of [Starsim](https://docs.starsim.org). It provides disease models (HIV, syphilis, chlamydia, gonorrhea, trichomonas, BV), structured sexual networks with risk groups, demographics, and interventions.

STIsim follows the Starsim philosophy: **common tasks should be simple**. Creating and running a simulation takes just a few lines of code.

## Hello world

The simplest way to create an STIsim simulation is to pass a disease name as a string:

In [ ]:
import stisim as sti

sim = sti.Sim(diseases='hiv')
sim.run()
sim.plot()

That's it. But a lot just happened behind the scenes. Let's unpack what's in the box.

## What's in the box

When you call `sti.Sim(diseases='hiv')`, STIsim creates a complete simulation with sensible defaults. Here's what gets set up automatically:

### Simulation parameters

The simulation runs from **2000 to 2030** with **monthly timesteps** and **1,000 agents** by default. You can inspect these:

In [ ]:
sim = sti.Sim(diseases='hiv')
sim.init()

print(f'Start:    {sim.pars.start}')
print(f'Stop:     {sim.pars.stop}')
print(f'Timestep: {sim.pars.dt} years ({sim.pars.dt*12:.0f} month)')
print(f'Agents:   {sim.pars.n_agents}')

### Networks

Two networks are created automatically:

- **StructuredSexual**: The main sexual contact network. Agents are assigned to risk groups (low, medium, high), form partnerships of different types (stable, casual, one-time, sex work), and have age-based partner preferences. See the [Networks](../networks/structured_sexual.md) documentation for details.
- **MaternalNet**: A mother-to-child transmission network (from Starsim).

In [ ]:
for name, nw in sim.networks.items():
    print(f'{name}: {type(nw).__name__}')

### Disease

The string `'hiv'` was looked up and converted into an `sti.HIV` module with all default parameters (initial prevalence of 5%, CD4-based progression, ART effects, etc.). See the [Diseases](../diseases/) documentation for the full parameter tables.

In [ ]:
hiv = sim.diseases.hiv
print(f'Disease: {type(hiv).__name__}')
print(f'Initial prevalence: {hiv.pars.init_prev}')
print(f'ART efficacy: {hiv.pars.art_efficacy}')

### Demographics

By default (without a location string), **no demographic modules** are created -- there are no births, deaths, or migration. This keeps the default sim fast and simple. To add demographics, either pass modules directly or provide a location string:

In [ ]:
print(f'Demographics: {dict(sim.demographics)}')
print(f'(Empty by default -- pass demographics="zimbabwe" to auto-load demographic data)')

## Customizing the simulation

There are multiple ways to configure a sim, depending on how much control you need.

### Passing parameters inline

The simplest way -- pass keyword arguments directly:

In [ ]:
sim = sti.Sim(
    diseases='hiv',
    n_agents=2000,
    start=2010,
    stop=2025,
)
sim.run()

### Passing disease-specific parameters

Use `sti_pars` to override disease defaults without creating the disease object yourself:

In [ ]:
sim = sti.Sim(
    diseases=['hiv', 'syphilis'],
    sti_pars=dict(
        hiv=dict(init_prev=0.1),
        syphilis=dict(init_prev=0.02),
    ),
)
sim.run()

### Using module instances

For full control, create the module objects directly. This is the standard Starsim pattern and gives you access to every parameter:

In [ ]:
import starsim as ss

hiv = sti.HIV(init_prev=0.1, art_efficacy=0.9)
nw = sti.StructuredSexual()
pregnancy = sti.Pregnancy(fertility_rate=20)
deaths = ss.Deaths(death_rate=10)

sim = sti.Sim(
    diseases=hiv,
    networks=nw,
    demographics=[pregnancy, deaths],
    n_agents=2000,
    start=2010,
    stop=2030,
)
sim.run()
sim.plot()

## Available diseases

STIsim includes the following disease modules, which can be passed by name or as instances:

| Name | Class | String alias | Type |
|------|-------|-------------|------|
| HIV | `sti.HIV` | `'hiv'` | CD4-based progression |
| Syphilis | `sti.Syphilis` | `'syphilis'` / `'syph'` | Staged (primary/secondary/latent/tertiary) |
| Chlamydia | `sti.Chlamydia` | `'ct'` | SEIS |
| Gonorrhea | `sti.Gonorrhea` | `'ng'` | SEIS |
| Trichomoniasis | `sti.Trichomoniasis` | `'tv'` | SEIS with persistence |
| Bacterial vaginosis | `sti.BV` | `'bv'` | CST-based |
| Genital ulcer disease | `sti.GUD` | `'gud'` | Simple SIS |

See the [Diseases](../diseases/) documentation for state diagrams and parameter tables for each disease.

## Exercises

1. Create a sim with both HIV and gonorrhea. Run it and plot the results.
2. Try changing the initial HIV prevalence to 20% and observe how it affects the epidemic trajectory.
3. Add demographics by passing `demographics='zimbabwe'` and compare the results to the default (no demographics) sim.